In [34]:
import cv2
import numpy as np
import pandas as pd
import os

dataset_path = '/home/jack/Mounts/DiskOne/kona_coffee/dataset/'

os.makedirs(dataset_path, exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'test', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'test', 'labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'train', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'train', 'labels'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'validate', 'images'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'validate', 'labels'), exist_ok=True)

df = pd.read_csv('validated.csv')

df = df.dropna()

df = df[df['valid'] == True]

df

,image,x,y,mask,valid
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,324,300,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,272,321,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,364,288,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
9,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,109,178,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
...,...,...,...,...,...
770,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,106,106,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
772,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,22,221,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
773,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,227,20,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
774,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,139,328,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True


In [35]:
def get_contour(row):
    mask = np.load(row['mask'])
    mask = mask.astype(np.uint8)
    
    width, height = mask.shape

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) <= 0:
        return

    largest_contour = max(contours, key=cv2.contourArea)
    
    # normalize contour
    largest_contour = largest_contour.astype(np.float32)
    largest_contour[:, :, 0] = largest_contour[:, :, 0] / width
    largest_contour[:, :, 1] = largest_contour[:, :, 1] / height
    
    return largest_contour


df['contour'] = df.apply(get_contour, axis=1)

df = df.dropna()

df = df[['contour', 'image']]

df

,contour,image
0,"[[[0.735, 0.065]], [[0.7325, 0.0675]], [[0.72,...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...
1,"[[[0.7925, 0.6925]], [[0.79, 0.695]], [[0.7825...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...
2,"[[[0.685, 0.75]], [[0.68, 0.755]], [[0.6775, 0...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...
3,"[[[0.855, 0.6775]], [[0.8525, 0.68]], [[0.8475...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...
9,"[[[0.2525, 0.395]], [[0.25, 0.3975]], [[0.2425...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...
...,...,...
770,"[[[0.2675, 0.205]], [[0.265, 0.2075]], [[0.26,...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...
772,"[[[0.1425, 0.465]], [[0.135, 0.4725]], [[0.132...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...
773,"[[[0.525, 0.0]], [[0.525, 0.045]], [[0.5275, 0...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...
774,"[[[0.34, 0.7475]], [[0.3375, 0.75]], [[0.335, ...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...


In [36]:
# set numpy seed
np.random.seed(42)

# set 70% of the data for training, 15% for testing, and 15% for validation
df['set'] = np.random.choice(['train', 'test', 'validate'], size=len(df), p=[0.7, 0.15, 0.15])

df

,contour,image,set
0,"[[[0.735, 0.065]], [[0.7325, 0.0675]], [[0.72,...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,train
1,"[[[0.7925, 0.6925]], [[0.79, 0.695]], [[0.7825...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,validate
2,"[[[0.685, 0.75]], [[0.68, 0.755]], [[0.6775, 0...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,test
3,"[[[0.855, 0.6775]], [[0.8525, 0.68]], [[0.8475...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,train
9,"[[[0.2525, 0.395]], [[0.25, 0.3975]], [[0.2425...",/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,train
...,...,...,...
770,"[[[0.2675, 0.205]], [[0.265, 0.2075]], [[0.26,...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...,validate
772,"[[[0.1425, 0.465]], [[0.135, 0.4725]], [[0.132...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...,train
773,"[[[0.525, 0.0]], [[0.525, 0.045]], [[0.5275, 0...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...,train
774,"[[[0.34, 0.7475]], [[0.3375, 0.75]], [[0.335, ...",/home/jack/Mounts/DiskOne/kona_coffee/augmente...,train


In [37]:
# print total train, test, and validate
print('total train:', len(df[df['set'] == 'train']))
print('total test:', len(df[df['set'] == 'test']))
print('total validate:', len(df[df['set'] == 'validate']))

total train: 356
total test: 75
total validate: 87


In [38]:
for group in df.groupby(['set', 'image']):
    sett = group[0][0]
    image_path = group[0][1]
    dfg = group[1]
    
    base_dir = os.path.join(dataset_path, sett)
    image_file_name = os.path.basename(image_path)
    label_file_name = os.path.splitext(image_file_name)[0] + '.txt'
    
    new_image_path = os.path.join(base_dir, 'images', image_file_name)
    label_path = os.path.join(base_dir, 'labels', label_file_name)
    
    # copy image_path to new_image_path
    if not os.path.exists(new_image_path):
        os.system('cp ' + image_path + ' ' + new_image_path)
    
    if not os.path.exists(label_path):
        with open(label_path, 'w') as label_file:
            # <class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
            label_file.write('0 ')
            for row in dfg.itertuples():
                contour = row.contour
                for point in contour:
                    label_file.write(str(point[0][0]) + ' ' + str(point[0][1]) + ' ')
            label_file.write('\n')

In [39]:
yaml_content = f'''
path: {dataset_path}
train: train/images
test: test/images
val: validate/images

names: ['bean']
'''
    
with open('dataset.yaml', 'w') as f:
    f.write(yaml_content)

In [40]:
# print tree of dataset
os.system('tree ' + dataset_path)

/home/jack/Mounts/DiskOne/kona_coffee/dataset/
├── test
│   ├── images
│   │   ├── 100_0001_3200_2000.png
│   │   ├── 100_0005_2800_1200.aug.2.png
│   │   ├── 100_0005_4400_2000.png
│   │   ├── 100_0006_1200_800.aug.1.png
│   │   ├── 100_0007_400_2000.aug.2.png
│   │   ├── 100_0008_1200_2800.png
│   │   ├── 100_0012_2000_2000.aug.0.png
│   │   ├── 100_0018_4000_400.aug.0.png
│   │   ├── 100_0019_2000_1200.aug.1.png
│   │   ├── 100_0019_2800_1600.aug.2.png
│   │   ├── 100_0019_3600_1200.png
│   │   ├── 100_0020_2400_1200.png
│   │   ├── 100_0020_400_3200.png
│   │   ├── 100_0021_3600_2000.png
│   │   ├── 100_0022_3600_3200.png
│   │   ├── 100_0023_1200_2000.png
│   │   ├── 100_0024_2800_2000.png
│   │   ├── 100_0025_1600_2000.aug.0.png
│   │   ├── 100_0025_2400_1600.png
│   │   ├── 100_0025_400_2000.png
│   │   ├── 100_0025_800_2400.png
│   │   ├── 100_0028_400_1600.aug.2.png
│   │   ├── 100_0029_2400_800.png
│   │   ├── 100_0030_2400_3200.png
│   │   ├── 100_0031_1600_1600.png
│   │   

0